In [1]:
import krpc
import csv
import math
import time

In [2]:
conn = krpc.connect()

#### Static Variable Definitions

In [3]:
vessel = conn.space_center.active_vessel
kerbin = conn.space_center.bodies['Kerbin']

In [4]:
# Reference frame relative to vessel, with z oriented towards center of kerbin
cframe = conn.space_center.ReferenceFrame.create_hybrid(kerbin.reference_frame,vessel.surface_reference_frame)

In [5]:
# Kerbin Gravitational Parameter
# Requires connection
GM = kerbin.gravitational_parameter #3.5316000e12

In [6]:
# Fuel flow rate in kilograms per second
# Requires connection
kgps = (vessel.max_thrust ) / (vessel.specific_impulse * 9.807)

ZeroDivisionError: float division by zero

In [8]:
# Test values for starting position and velocity
tspos = (604608.6391293579, -1.318767317570746e-11, 1.1641532182693481e-10)
tsvel = (179.7520206477589, 6.886157014239632, 76.08919378846531)

#### Functions

In [6]:
# Reads variables from APTD.csv and assigns to lists
# APTD.csv must be in current working directory

logsheet = open("APTD.csv",'rb')
reader = csv.reader(logsheet)
variables = []
reader.next()
for row in reader:
    variables.append(row)
logsheet.close()

variables = [list(x) for x in zip(*variables)]

altL   = [float(x) for x in variables[0]]
pressL = [float(x) for x in variables[1]]
tempL  = [float(x) for x in variables[2]]
densL  = [float(x) for x in variables[3]]

del variables

In [7]:
# Takes altitude
# Returns static atmospheric pressure at given altitude
def altPress(altitude):
    i = [abs(altitude-x) for x in altL].index(min([abs(altitude-x) for x in altL]))
    return pressL[i]

In [8]:
# Takes altitude
# Returns air density (rho) at given altitude
def rho(altitude):
    i = [abs(altitude-x) for x in altL].index(min([abs(altitude-x) for x in altL]))
    return densL[i]

In [9]:
## Thrust = Vac Thrust -((Vac Thrust - Sea Level Thrust)*(local press / sea lvl press))

# Takes altitude in meters
# Optionally takes vacThrust (vessel max thrust in vaccum - retrievable with vesse.max_vacuum_thrust)
# Optionally takes tRange (Thrust range as vacThrust - sea level thrust)
# Optionally takes and seaPress (Sea Level atmospheric pressure in Pa)
# Returns thrust at given altitude (in Newtons I'm assuming)
# Requires altPress
def Thrust(alt, vacThrust = 215000.0, tRange = 46578.78125, seaPress = 100142.2578125):
    return  vacThrust - ( tRange * ( altPress(alt) / seaPress ) )

In [10]:
# Reverse Thrust Vector
# Takes a velocity vector, altitude, and vessel mass 
# Returns thrust acceleration as a vector in the opposite direction
def RTV(vvec, alt, mass):
    def axis(x):
        return ( Thrust(alt) * (vvec[x]/sum(vvec)) ) / mass
    return (-axis(0), -axis(1), -axis(2) )

In [11]:
# Returns acceleration due to gravity
# Requires connection
def agrav(dist):
	return GM / (dist ** 2)

In [12]:
# Takes altitude and velocity
# Optionally takes CdA and mass
# Returns acceleration due to drag
# Requires rho()
def adrag(alt, vel, mass = 5241.39, CdA = 1.3):
	return (rho(alt) * vel**2 * .5 * CdA) / mass

In [13]:
# Takes two coordiante lists
# Returns distance betwen two coordinates (???)
def hyp(x,y):
    return math.sqrt(abs(x[0]-y[0])**2 + abs(x[1]-y[1])**2 + abs(x[2]-y[2])**2)

In [14]:
# Takes a position coordinate
# Returns distance from the origin of that coordinate
def dfo(pos):
    return math.sqrt(abs(pos[0])**2 + abs(pos[1])**2 + abs(pos[2]**2))

### Loop

In [160]:
# Return a list containing 2x lists of 3x coordinate tuples, representing position and velocity at one second increments until impact
# def freefalltrajectory():
    # Start with current vessel position and velocity
pos = vessel.position(cframe) # coordinate tuple
vel = vessel.velocity(cframe) # coordinate tuple
trajectory = []
trajectory.append([pos,vel]) # a list of 2x lists of 3x coordinate tuples

while pos[0] > 600100:
    pos = (pos[0]+vel[0],pos[1]+vel[1],pos[2]+vel[2])
    vel = (vel[0] - agrav(pos[0]) + adrag(pos[0]-600000,vel[0]), vel[1] - adrag(pos[0]-600000,vel[1]), vel[2] - adrag(pos[0]-600000,vel[2]) )
    trajectory.append([pos,vel])
    
if trajectory[-1][0][0] < 600000:
    del trajectory[-1]

trajectory

[[(632217.4681666809, 1.318767317570746e-11, 1.7462298274040222e-10),
  (-632.7663422875218, 24.976337560899772, 289.0844518869077)],
 [(631584.7018243934, 24.97633756091296, 289.0844518870823),
  (-641.4082753454634, 24.97600814935161, 289.0403222069449)],
 [(630943.2935490479, 49.95234571026457, 578.1247740940272),
  (-650.0335542349312, 24.975635003403575, 288.990347582336)],
 [(630293.259994813, 74.92798071366815, 867.1151216763633),
  (-658.6367889482642, 24.975212135209475, 288.9337316559546)],
 [(629634.6232058648, 99.90319284887762, 1156.0488533323178),
  (-667.2117485130597, 24.97473284299294, 288.8695843387069)],
 [(628967.4114573518, 124.87792569187056, 1444.9184376710245),
  (-675.7513462617189, 24.974189768130927, 288.79692998714285)],
 [(628291.66011109, 149.85211546000147, 1733.7153676581675),
  (-684.2476326212226, 24.973574954781842, 288.71471601753444)],
 [(627607.4124784688, 174.8256904147833, 2022.430083675702),
  (-692.6917945150901, 24.97287990958969, 288.62182163

### Scratch

In [40]:
# Return a list containing 2x lists of 3x coordinate tuples, representing position and velocity at one second increments until impact
def trajectory(pos = vessel.position(cframe), vel = vessel.velocity(cframe), harddeck = 600100):
    # Start with current vessel position and velocity
    trajectory = []
    trajectory.append([pos,vel]) # a list of 2x lists of 3x coordinate tuples

    while pos[0] > harddeck:  #TODO:  transform pos[0] to kerbin reference frame to get altitiude of predicted point
        pos = (pos[0]+vel[0],pos[1]+vel[1],pos[2]+vel[2])
        vel = (vel[0] - agrav(pos[0]) + adrag(pos[0]-600000,vel[0]), vel[1] - adrag(pos[0]-600000 - Thrust(vel[1]), vel[2] - adrag(pos[0]-600000,vel[2]) )
        trajectory.append([pos,vel])

    if trajectory[-1][0][0] < harddeck:
        del trajectory[-1]

    return trajectory

SyntaxError: invalid syntax (<ipython-input-40-c2b278512497>, line 9)

In [ ]:
x,y,z = RTV(pos[0]-600000,)

In [ ]:
trajectory = trajectory()
for tick in enumerate(trajectory[0][::-1]):
    
    

In [43]:
vessel.position(cframe)

(603020.1701079655, 5.701394911739044e-11, -6.402842700481415e-10)

In [19]:
vessel.velocity(cframe)

(-29.601089182009392, 0.8279541412193142, -0.06489352897457934)

In [2]:
range(10)[::-1]

[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

In [60]:
ap = vessel.auto_pilot
ap.engage()

In [4]:
ap.target_pitch_and_heading(70,270)

NameError: name 'ap' is not defined

In [82]:
ISP = []
Thrust = []
mass = []
itime = []

In [83]:
# Prep control systems for launch
vessel.control.sas = False
vessel.control.rcs = False
vessel.control.throttle = 1.0

# Launch and fly towards zenith
vessel.control.activate_next_stage()
vessel.auto_pilot.engage()
vessel.auto_pilot.target_pitch_and_heading(90, 90)

while vessel.flight().surface_altitude < 40000:
    ISP.append(vessel.specific_impulse)
    Thrust.append(vessel.thrust)
    mass.append(vessel.mass)
    itime.append(time.time())
    
# Confirm loop completion and set throttle to 0
print "loop complete"
vessel.control.throttle = 0.0
vessel.auto_pilot.disengage()

loop complete


In [ ]:
# vel = (vel[0] - agrav(pos[0]) + adrag(pos[0]-600000,vel[0]), vel[1] - adrag(pos[0]-600000 - Thrust(vel[1]), vel[2] - adrag(pos[0]-600000,vel[2]) )
def updateV(dimension, val):
    if dimension = x:
        val 
        
        